In [1]:
# import the socket library provided by Python
# theoretically, you can only use the socket library to create an Internet server/client
# however, the socket library is a low-level library
# it may not easy to use socket library only to create complicated network-based apps

# if you are interested in all functions provided by Python, see this
# https://docs.python.org/3.7/library/socket.html

import socket
# As a server, we would like to serve multiple clients
# so we use one thread to serve one client
# threading is a python library that can spawn multiple threads in a single app (process)

import threading

# create socket instance
# socket.AF_INET is a constant value that indicates I want to use IP (Internet Protocol) as my L3 protocol
# socket.SOCK_STREAM is a constant value that indicates I want to use TCP as my L4 protocol

s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

# ask the OS to bind the created socket to user-specified parameters: (IP, TCP port)
# the 1st parameter is the binded IP, the 2nd parameter is the binded port number
# So that when the OS receives a datagram, the OS knows how to demux the datagram to the corresponding application 

pars = ('127.0.0.1', 7777) # you can change the server port to whatever you want
s.bind(pars)

# become a server socket
# it makes this python program waiting for receiving message
# listen() function has one parameter that limits how many clients can be connected to this server
# we set it to 5 without any reason
# you can change it to any number you want, as long as you have sufficient resources (computing, memory)

s.listen(5)

# a new thread is created for every new accepted client
# every new thread starts from the function below



In [2]:
from urllib.parse import unquote, quote

# this class is copy from others library that already on the internet
# in order to make py homework execute simpler and easier, I only copy the code that I need this time
class Request:
    def __init__(self, r):
        self.content = r
        self.method = r.split()[0]
        self.path = r.split()[1]
        self.body = r.split('\r\n\r\n', 1)[1]

    def form_body(self):
        return self._parse_parameter(self.body)

    def parse_path(self):
        index = self.path.find('?')
        if index == -1:
            return self.path, {}
        else:
            path, query_string = self.path.split('?', 1)
            query = self._parse_parameter(query_string)
            return path, query

    # the method to access header porperty
    @property
    def headers(self):
        header_content = self.content.split('\r\n\r\n', 1)[0].split('\r\n')[1:]
        result = {}
        for line in header_content:
            k, v = line.split(': ')
            result[quote(k)] = quote(v)
        return result

    # the method only use in this class 
    @staticmethod
    def _parse_parameter(parameters):
        args = parameters.split('&')
        query = {}
        for arg in args:
            k, v = arg.split('=')
            query[k] = unquote(v)
        return query

In [3]:
import os
def response_for_request(request):
    # make the absolute path of the directory
    path = os.getcwd()
    path += request.parse_path()[0]
    print(path)

    # send different response refer to different homework request
    if request.parse_path()[0] == "/redirect.html":
        return "HTTP/1.1 301 Moved Permanently\nContent-Type: text/html\nLocation: /good.html\n\n".encode('utf-8')
    elif request.parse_path()[0] == "/good.html":
        return "HTTP/1.1 200 OK\nContent-Type: text/html\nContent-Length: 106\n\n<html><head><link href=\"/style.css\" rel=\"stylesheet\" type=\"text/css\"></head><body>good</body></html>".encode('utf-8')
    elif request.parse_path()[0] == "/style.css":
        return "HTTP/1.1 200 OK\nContent-Type: text/css\nContent-Length: 18\n\nBody {color: red;}".encode('utf-8')
    else:
        return "HTTP/1.1 404 Not Found\nContent-Type: text/html\nContent-Length: 51\n\n<html><body>Error 404: File not found</body></html>".encode('utf-8')

In [4]:
def serveClient(clientsocket, address):
    
    # we need a loop to continuously receive messages from the client
    while True:
        # then receive at most 1024 bytes message and store these bytes in a variable named 'data'
        # you can set the buffer size to any value you like
        data = clientsocket.recv(1024).decode()
        print("from client", data)
        
        # if the received data is not empty, then we send something back by using send() function
        # use the response_for_request function
        if data:
            data2 = Request(data)
            print(response_for_request(data2))
            clientsocket.send(response_for_request(data2))
            

In [ ]:
# since at most we can serve many clients (5 in this example), we need a way to distinguish them 
# as mentioned in the class, TCP use 4-tuple (src IP, dst IP, src port, dst port) to distinguish a socket
# we use accept() function to confirm that we connect to the client socket
# and accept() function will return the client's socket instance and IP
# we need a loop to keep accepting new clients (until 5 clients are accepted)

while True:
    # accept a new client and get it's information
    (clientsocket, address) = s.accept()
    
    # create a new thread to serve this new client
    # after the thread is created, it will start to execute 'target' function with arguments 'args' 
    threading.Thread(target = serveClient, args = (clientsocket, address)).start()